In [1]:
from dash import html, dcc, callback, Output, Input, State
from jupyter_dash import JupyterDash
import pandas as pd
import plotly.express as px

In [2]:
url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv'
data = pd.read_csv(url)

In [3]:
data = data[data["continent"].notna()]
data['date'] = pd.to_datetime(data['date'])
data = data.fillna(0)

In [4]:
dates = pd.to_datetime(data['date'].unique())
marks = pd.date_range(start=dates[0],end=dates[-1],freq="MS")

In [5]:
from itertools import compress

In [23]:
marks_dict = {i : str(yearmon) for i, yearmon in enumerate(dates.strftime('%Y-%m-%d'))}
mark_keys= list(marks_dict.fromkeys((list(compress(marks_dict.keys(),dates.is_month_start)))).keys())[::2]
marks={key: marks_dict[key] for key in mark_keys}

In [13]:
continents= data["continent"].unique().tolist()
continents.append("World")
continents.remove("Oceania")

In [25]:
def make_plot(df,col,cont):
    fig = px.scatter_geo(df,
                         locations="iso_code",
                         size=col,
                         scope=cont.lower(),
                         size_max=40,
                         height=800,
                         color="continent",
                         hover_name='location',
                         hover_data={'continent':False,'iso_code':False},
                         )
    fig.update_geos(showcountries=True)
    fig.update_layout(
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Times New Roman"
            )
        )
    return fig

In [27]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div(children='Világtérkép a COVID-19-ről', style={'textAlign': 'center', 'fontSize': 48}),
    html.Hr(),
    html.Div(
    [
        html.Label(children='Válasszon egy opciót:'),
        dcc.Dropdown(id='dropdown_col',
                options=[
                    {'label': 'Összes eset', 'value': 'total_cases'},
                    {'label': 'Összes haláleset', 'value': 'total_deaths'},
                    {'label': 'Összes eset 1 millió főre vetítve', 'value': 'total_cases_per_million'},
                    {'label': 'Összes haláleset 1 millió főre vetítve', 'value': 'total_deaths_per_million'}],
                value='total_cases'),
        html.Br(),
        html.Label(children='Válasszon egy földrészt:'),
        dcc.Dropdown(id='dropdown_cont',options=continents,
                    value="World")
    ],style={'width':'30%'}),
    dcc.Slider(min=0,
            max=len(marks_dict),
            value=0,
            marks=marks,
            step=1,
            id='dateslider'),
    html.Div(id='date_label',style={'fontSize': 24}),
    html.Div(id='col_label',style={'fontSize': 24}),
    dcc.Graph(id='graph_sc_geo',figure={}),
])


@callback(
    Output('graph_sc_geo', 'figure'),
    Output('data_label','children'),
    Output('col_label','children'),
    Input('dropdown_col', 'value'),
    Input('dateslider','value'),
    Input('dropdown_cont','value'),
    State('dropdown_col','options')
)
def update_graph(col,slider_value,cont,dropdown_label):
    df = data[data['date']==marks_dict[slider_value]]
    data_label= f'Dátum: {marks_dict[slider_value]}'
    label = [x['label'] for x in dropdown_label if x['value']==col][0]
    label_value = int(df[col].sum()) if cont == 'World' else int(df[df['continent']==cont][col].sum())
    col_label = f'{label}: {label_value:,}'
    fig = make_plot(df,col,cont)
    return fig,data_label,col_label

                                                 
app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/

